In [1]:
import json
import math
import pickle
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

## 一.瓜子车源数据匹配

In [2]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# reload(sys)
# sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
# PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
# PRODUCE_DB_USER = 'pingjia'
# PRODUCE_DB_PASSWD = 'De32wsxC'

PRODUCE_DB_ADDR_OUTTER = '101.201.143.74'
PRODUCE_DB_USER = 'leidengjun'
PRODUCE_DB_PASSWD = 'ldj_DEV_~!'

PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'
PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://' + PRODUCE_DB_USER + ':' + PRODUCE_DB_PASSWD + '@' + PRODUCE_DB_ADDR_OUTTER + '/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_CHINA_USED_CAR_ESTIMATE_ENGINE)

yesterday = datetime.datetime.now() - datetime.timedelta(days=1)
start = str(yesterday.year) + '-' + str(yesterday.month) + '-' + str(yesterday.day) + ' 00:00:00'
today = datetime.datetime.now()
end = str(today.year) + '-' + str(today.month) + '-' + str(today.day) + ' 00:00:00'

query_sql = 'select * from crawler_car_source ' \
            'where domain != \'xin.com\' and create_time >= \''+start+'\' and  create_time < \''+end+'\' '

# query_sql = 'select detail_name,year,month,mile,price,city,domain from crawler_car_source where create_time >= \'2019-03-16\' '
origin_train = pd.read_sql_query(query_sql, engine)
origin_train.to_csv('./origin_train.csv', index=False)
print('finish query, num:'+str(len(origin_train)))

finish query, num:9479


### 一.车源更新脚本

In [3]:
# coding=utf-8
import os
import re
import json
import math
import pickle
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """
    text = df[column_name]
    text = text.lower()
    text = text.replace('ⅰ', 'i')
    text = text.replace('ⅱ', 'ii')
    text = text.replace('ⅲ', 'iii')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = text.replace('【', '')
    text = text.replace('】', '')
    text = text.replace('（', '')
    text = text.replace('）', '')
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace('_', '')
    text = text.replace('-', '')
    text = text.replace('+', '')
    text = text.replace('—', '')
    text = text.replace('/', '')
    text = text.replace('“', '')
    text = text.replace('”', '')
    text = text.replace('!', '')
    text = text.replace('。', '')
    text = text.replace('＞', '')
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('》', '')
    text = text.replace('！', '')
    text = text.replace('／', '')
    text = text.replace('’', '')
    text = text.replace('－', '')
    text = text.replace('•', '')
    text = text.replace('×', '')
    text = text.replace('《', '')
    text = text.replace('＿', '')
    text = text.replace('®', '')
    text = text.replace('─', '')
    text = text.replace('、', '')
    text = text.replace(' ', '') 
    return text

def process_url(df):
    return df['url'].replace('https://','')

def process_eval_pirce(df):
    random_price = random.randint(int(-df['price']*0.95*0.03*10000),int(df['price']*0.95*0.03*10000))
    if (df['price'] * 0.95 + random_price/10000) < 0.5:
        return df['price']
    return float('%.2f' % (df['price'] * 0.95 + random_price/10000))

def process_gpj_index(df):
    k = 2
    if df['eval_price'] < 0.5:
        df['eval_price'] = 0.5
    b = abs(df['price'] * 0.95 - df['eval_price']) / df['eval_price']
    return pd.Series([float('%.2f' % (10 * (math.e ** (-k*b)))),int((10 * (math.e ** (-k*b)))*100000000)])

def process_thumbnail(df):
    url = []
    thumbnail = json.loads(df['thumbnail'])
    if df['domain'] == 'guazi.com':
        if len(thumbnail) < 10:
            return pd.Series([np.NaN,np.NaN])
        url.append(thumbnail[0]['images'][0]['image'])
        url.append(thumbnail[1]['images'][0]['image'])
        url.append(thumbnail[5]['images'][0]['image'])
        url.append(thumbnail[6]['images'][0]['image'])
        url.append(thumbnail[7]['images'][0]['image'])
        url.append(thumbnail[8]['images'][0]['image'])
        url.append(thumbnail[9]['images'][0]['image'])
        url.append(thumbnail[10]['images'][0]['image'])
        return pd.Series([thumbnail[0]['images'][0]['image'],url])
    else:
        url.append(thumbnail[0]['images'][0]['image'])
        url.extend(thumbnail[1]['images'][0]['image'].split(' '))
        return pd.Series([thumbnail[0]['images'][0]['image'],url])
    return pd.Series([np.NaN,np.NaN])

def add_brand(brand):
    brand = brand.append(pd.DataFrame([['昌河','79'],['吉利','25'],['传祺','82'],['swm斯威','269'],['斯威','269'],['猎豹','78'],
                                      ['力帆','80'],['五菱','114'],['野马','111'],['mg','20'],['英致','192'],['汉腾','267'],
                                      ['吉奥','108'],['康迪','219'],['夏利','110'],['莲花','89'],['北汽','154']],columns=['brand_name','brand_slug']),sort=False).reset_index(drop=True)
    return brand

def find_brand(df):
    for i in range(0,len(brand)):
        if brand.loc[i,'brand_name'] == 'mg':
            match = re.findall('^mg',df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
        else:           
            match = re.findall(brand.loc[i,'brand_name'],df['detail_name'])
            if len(match) > 0:
                return pd.Series([brand.loc[i,'brand_name'],brand.loc[i,'brand_slug']])
    return pd.Series([np.NaN,np.NaN])

def process_online_year(df):
    temp = re.findall('(\d\d\d\d)款',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_volume(df):
    temp = re.findall('(\d\.\d)',df['detail_name'])
    if len(temp) == 0:
        return np.NaN
    return temp[0]

def process_control(df):
    text = df['detail_name']
    text = text.lower()
    temp = re.findall('自动|手自|amt|无级|cvt|双离合|dct|dsg',text)
    if len(temp) > 0:
        return '自动'
    temp = re.findall('手动',text)
    if len(temp) > 0:
        return '手动'
    return np.NaN

def cal_brand_count(df):
    return len(df['brand_name'])

def cal_cos_similar(df,y_vector):
    x_vector = eval(df['fill_vector'])
    y_vector = eval(y_vector)
    return np.dot(x_vector,y_vector) / (np.linalg.norm(x_vector,ord=2)*np.linalg.norm(y_vector,ord=2) )

def match_detail(df):
    print(df['brand_slug'],df['online_year'],df['control'],df['volume'])
    if (type(df['control']) != str) & math.isnan(float(df['volume'])):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
    elif (type(df['control']) != str) & (not math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    elif (type(df['control']) == str) & (math.isnan(float(df['volume']))):
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control']),:].reset_index(drop=True)
    else:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year']))&(car_autohome_all['control'] == df['control'])&(car_autohome_all['volume'] == float(df['volume'])),:].reset_index(drop=True)
    
    if len(temp_detail) == 0:
        temp_detail = car_autohome_all.loc[(car_autohome_all['brand_slug'] == int(df['brand_slug']))&(car_autohome_all['online_year'] == int(df['online_year'])),:].reset_index(drop=True)
        if len(temp_detail) == 0:
            return pd.Series([np.NaN,np.NaN,np.NaN,np.NaN,np.NaN])
    temp_detail['cos_similar'] = temp_detail.apply(cal_cos_similar,args=(df['fill_vector'],),axis=1)
    temp_detail = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), :]
    result = temp_detail.loc[temp_detail.groupby(['brand_slug']).cos_similar.idxmax(), ['model_slug','model_name','detail_slug','detail_name','cos_similar']].values[0]
    return pd.Series(result)

#########################################step1
car_autohome_all = pd.read_csv('./car_autohome_cos_vector.csv')

pcm = pd.read_csv('./province_city_map.csv')
pcm = pcm.loc[:,['city','province']]

gpj_model = pd.read_csv('./combine_model.csv',low_memory=False)
gpj_model = gpj_model.loc[:,['slug','parent','classified_slug','popular']]
gpj_model = gpj_model.rename(columns={'slug':'model_slug','parent':'brand_slug','classified_slug':'body_style'})

gpj_detail = pd.read_csv('./combine_detail.csv',low_memory=False)
gpj_detail = gpj_detail.loc[:,['global_slug','volume','control','car_autohome_detail_id','detail_model_slug']]
gpj_detail = gpj_detail.rename(columns={'global_slug':'model_slug','car_autohome_detail_id':'detail_slug'})
gpj_detail = gpj_detail.merge(gpj_model,how='left',on=['model_slug'])
gpj_detail = gpj_detail.drop_duplicates(['detail_slug','volume','control']).reset_index(drop=True)

##########################################step2
origin_train = pd.read_csv('./origin_train.csv')

origin_train = origin_train.loc[(origin_train['detail_name'].notnull())&(origin_train['year'].notnull()),:].reset_index(drop=True)
origin_train['online_year'] = origin_train.apply(process_online_year,axis=1)
origin_train = origin_train.loc[(origin_train['online_year'].notnull()),:].reset_index(drop=True)
origin_train['volume'] = origin_train.apply(process_volume,axis=1)
origin_train['control'] = origin_train.apply(process_control,axis=1)
origin_train['title'] = origin_train['detail_name']
origin_train['detail_name'] = origin_train.apply(delete_str_useless,args=('detail_name',),axis=1)

brand = car_autohome_all.loc[:,['brand_name','brand_slug']].drop_duplicates(['brand_name','brand_slug']).reset_index(drop=True)
brand = add_brand(brand)
brand['name_count'] = brand.apply(cal_brand_count,axis=1)
brand = brand.sort_values(by=['name_count'],ascending=False).reset_index(drop=True)
model = car_autohome_all.loc[:,['brand_name','brand_slug','model_name','model_slug']].drop_duplicates(['brand_name','brand_slug','model_name','model_slug']).reset_index(drop=True)
origin_train[['brand_name','brand_slug']] = origin_train.apply(find_brand,axis=1)
origin_train = origin_train.loc[(origin_train['brand_slug'].notnull()),:].reset_index(drop=True)
# origin_train.info()

f = open('./word_index.txt', 'r', encoding='UTF-8')
temp = f.read()
word_index = eval(temp)

with open('./tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

test_final = []
for i in range(0, len(origin_train)):
    temp = []
    x = origin_train['detail_name'][i]
    temp.extend(x)
    test_final.append(temp)
test_sequences = tokenizer.texts_to_sequences(test_final)

fill_zero = [0 for i in range(0,len(word_index))]

for i in range(0,len(origin_train)):  
    text = fill_zero.copy()
    for j in test_sequences[i]:
        text[j-1] = text[j-1] + 1
    origin_train.loc[i,'fill_vector'] = str(text)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})

origin_train[['model_slug','model_name','detail_slug','detail_name','cos_similar']] = origin_train.apply(match_detail,axis=1) 
origin_train = origin_train.loc[(origin_train['cos_similar'] >= 0.82),:].reset_index(drop=True)
origin_train = origin_train.drop(['fill_vector','cos_similar','control','volume','online_year'],axis=1)

fill_columns = car_autohome_all.loc[:,['detail_slug','online_year','price_bn','volume','control']]
cur_time = datetime.datetime.now()
car_source = origin_train.drop(['id','car_application','labels','title','brand_slug','model_slug','brand_name','model_name','detail_name'],axis=1)
car_source['pub_time'] = car_source['create_time']
car_source['status'] = 'sale'
car_source.loc[(car_source['domain'].isin(['guazi.com','xin.com'])),'source_type'] = 'cpersonal'
car_source.loc[(car_source['domain'].isin(['58.com','baixing.com'])),'source_type'] = 'personal'
car_source['process_status'] = 'S'
car_source['global_sibling'] = 0
car_source['city_sibling'] = 0
car_source['dealer_sibling'] = 0
car_source['domain_sibling'] = 0
car_source['qs_tags'] = '质保'
car_source['expired_at'] = datetime.datetime.now() + datetime.timedelta(days=10)
car_source['domain_sibling'] = 0
car_source['created_on'] = cur_time 
car_source['cs_clean'] = 0
car_source = car_source.merge(pcm, how='left', on=['city'])
car_source = car_source.merge(fill_columns, how='left', on=['detail_slug'])
car_source = car_source.merge(gpj_detail,how='left',on=['detail_slug','volume','control'])
car_source = car_source.rename(columns={'origin_name':'title','detail_model_slug':'model_detail_slug','popular':'popularity'})
car_source = car_source.drop(['create_time','detail_slug'],axis=1)
car_source = car_source.loc[(car_source['model_detail_slug'].notnull()),:].reset_index(drop=True)
car_source = car_source.drop_duplicates(['url'])
car_source['url'] = car_source.apply(process_url, axis=1)
car_source['eval_price'] = car_source.apply(process_eval_pirce, axis=1)
car_source[['gpj_index','rank']] = car_source.apply(process_gpj_index, axis=1)
car_source[['thumbnail','car_image']] = car_source.apply(process_thumbnail, axis=1)
car_source = car_source.loc[(car_source['car_image'].notnull()),:].reset_index(drop=True)
car_source = car_source.sort_values(by=['url','pub_time'],ascending=[True,False]).reset_index(drop=True)
car_source = car_source.drop_duplicates(['url']).reset_index(drop=True)
car_source.to_csv('./finish_match.csv',index=False)

13 2012 nan 2.0
13 2012 nan 2.0
3 2012 自动 2.5
62 2013 自动 1.6
12 2014 手动 1.4
36 2018 nan 2.0
1 2016 手动 1.6
71 2012 手动 1.6
49 2010 nan 3.0
63 2017 自动 2.0
154 2018 nan 2.0
25 2009 nan 1.5
53 2013 nan 1.4
1 2017 自动 nan
1 2010 自动 1.8
1 2012 手动 1.6
81 2015 手动 1.5
8 2012 自动 1.6
86 2015 自动 1.6
44 2015 nan 3.0
67 2018 自动 nan
1 2017 手动 1.6
14 2009 自动 1.8
36 2017 nan nan
12 2009 手动 1.8
86 2011 手动 1.6
15 2018 nan nan
14 2009 自动 1.8
36 2011 nan nan
56 2010 自动 1.0
15 2017 nan nan
8 2013 手动 1.5
3 2014 自动 1.6
75 2018 自动 1.5
1 2016 nan 1.8
3 2012 nan 2.7
1 2013 手动 1.4
77 2013 手动 1.5
8 2013 自动 1.8
1 2015 自动 1.4
36 2011 nan nan
25 2015 手动 1.5
67 2017 自动 1.6
8 2012 自动 1.8
95 2014 手动 2.0
15 2013 nan nan
181 2014 自动 2.4
114 2018 nan 1.5
52 2014 nan nan
33 2007 自动 2.0
3 2013 nan 2.5
269 2016 手动 1.8
15 2000 nan nan
71 2017 手动 1.5
36 2018 nan nan
15 2008 nan 2.5
63 2012 自动 2.0
1 2013 手动 1.4
3 2006 自动 2.5
33 2012 nan nan
67 2016 手动 1.6
71 2009 自动 1.6
58 2016 nan nan
8 2012 nan 1.8
33 2017 nan nan
3 2013 自动 2.0


38 2018 nan nan
1 2011 自动 1.6
1 2009 自动 1.8
71 2009 自动 1.8
14 2009 自动 1.8
14 2009 自动 1.8
14 2009 自动 1.8
14 2015 nan 2.4
1 2004 手动 1.8
173 2016 手动 1.5
13 2013 手动 1.6
72 2011 手动 1.6
14 2008 自动 2.0
12 2011 自动 1.4
63 2018 nan 2.0
14 2009 自动 1.8
26 2016 手动 2.0
12 2016 自动 1.4
12 2009 nan 1.6
154 2015 手动 2.0
67 2017 自动 1.6
8 2016 nan nan
14 2009 自动 1.8
14 2009 自动 1.8
13 2009 手动 1.4
53 2015 手动 1.4
14 2009 自动 1.8
14 2009 自动 1.8
12 2011 手动 1.6
14 2009 自动 1.8
76 2009 nan 1.3
70 2017 nan nan
14 2009 自动 1.8
14 2009 自动 1.8
3 2013 手动 1.6
14 2009 自动 1.8
12 2013 自动 2.0
77 2013 手动 1.5
3 2011 nan 2.7
62 2006 自动 1.6
14 2009 自动 1.8
12 2016 自动 1.6
14 2009 自动 1.8
14 2009 自动 1.8
14 2009 自动 1.8
14 2009 自动 1.8
14 2009 自动 1.8
3 2010 nan 2.5
82 2017 nan nan
38 2017 自动 1.5
41 2014 nan 2.4
95 2013 手动 1.6
38 2015 自动 1.5
33 2017 nan 2.0
120 2017 手动 1.5
76 2018 手动 1.6
12 2014 自动 1.4
14 2016 自动 1.5
114 2015 手动 1.2
3 2009 自动 1.6
22 2012 自动 1.6
3 2009 自动 1.6
120 2017 手动 1.5
3 2009 自动 1.6
14 2015 nan 2.4
3 2009 自动 1.6
120

33 2017 nan nan
33 2017 nan nan
63 2014 自动 1.6
3 2014 自动 1.6
14 2017 自动 1.5
12 2014 自动 1.8
8 2012 自动 1.6
63 2012 自动 1.6
3 2012 nan 2.5
1 2015 自动 2.0
38 2011 nan 1.6
38 2015 手动 1.4
120 2015 手动 1.8
15 2017 nan nan
73 2015 nan 2.0
38 2018 自动 nan
71 2013 手动 1.2
8 2013 自动 1.8
3 2017 自动 1.2
63 2012 手动 1.6
3 2013 自动 1.6
1 2011 手动 1.4
3 2017 自动 nan
14 2017 自动 1.5
3 2016 自动 1.6
259 2017 自动 1.5
13 2010 自动 2.0
63 2011 手动 1.5
53 2015 自动 1.6
14 2015 nan 2.0
14 2016 自动 1.5
3 2013 自动 2.5
33 2004 自动 1.8
3 2013 自动 2.5
38 2015 自动 1.5
72 2013 自动 1.6
76 2017 手动 1.6
53 2015 自动 1.5
52 2007 nan nan
38 2013 手动 1.5
71 2017 nan nan
15 2011 自动 2.5
72 2012 自动 2.3
3 2007 自动 2.4
14 2017 nan nan
45 2015 自动 1.0
3 2016 自动 1.6
12 2012 自动 1.6
8 2011 自动 1.8
122 2012 自动 1.6
203 2016 手动 1.5
67 2014 自动 1.4
12 2013 自动 2.0
8 2015 自动 nan
1 2018 自动 nan
1 2013 手动 1.6
20 2012 自动 1.8
36 2017 nan nan
3 2012 nan 2.7
12 2011 自动 1.6
56 2012 nan 1.6
1 2011 手动 1.4
114 2012 nan 1.5
36 2010 nan nan
86 2014 手动 1.6
263 2013 自动 2.4
81 2015 手

58 2014 自动 1.5
22 2016 自动 1.5
71 2009 nan 1.6
45 2016 nan 0.9
63 2014 自动 1.6
13 2014 自动 1.6
56 2012 手动 1.0
58 2010 手动 2.0
3 2005 自动 1.8
120 2017 手动 1.5
14 2016 自动 2.4
14 2015 自动 1.5
12 2011 手动 1.6
1 2011 nan 3.0
58 2013 自动 2.0
36 2012 nan nan
8 2015 自动 1.5
33 2012 自动 2.5
71 2012 手动 1.8
71 2017 自动 nan
3 2012 自动 2.5
1 2015 自动 1.6
33 2013 自动 2.0
76 2015 手动 1.4
15 2008 自动 2.5
38 2012 nan 2.4
71 2013 nan 1.4
14 2013 自动 1.8
3 2010 自动 2.0
67 2017 自动 1.6
58 2017 nan nan
82 2017 手动 1.3
27 2016 自动 1.5
3 2017 自动 1.6
1 2011 nan 1.8
38 2011 nan 2.4
96 2017 自动 2.0
14 2016 自动 2.0
33 2019 nan nan
15 2018 自动 2.0
1 2006 手动 1.8
14 2013 自动 1.8
1 2008 手动 1.6
1 2018 自动 nan
77 2017 nan nan
63 2014 手动 1.6
33 2018 nan nan
13 2011 手动 1.4
12 2014 自动 1.8
71 2009 nan 1.6
70 2019 nan nan
142 2013 手动 1.0
3 2007 自动 2.4
38 2006 自动 2.4
76 2016 手动 2.0
12 2015 自动 1.6
3 2014 手动 1.5
59 2005 nan nan
15 2011 nan nan
1 2011 自动 2.0
110 2011 手动 1.3
26 2006 手动 1.6
38 2008 自动 1.6
110 2011 手动 1.3
1 2002 自动 1.8
1 2002 自动 1.4
63 200

1 2016 nan 1.8
1 2014 自动 1.6
12 2013 自动 2.4
76 2017 自动 1.5
76 2016 自动 1.8
8 2013 nan 1.5
33 2010 nan 2.0
173 2015 手动 1.5
25 2015 nan 1.8
46 2012 nan 2.4
52 2010 nan nan
3 2011 自动 2.4
8 2011 手动 1.8
63 2011 nan 2.5
208 2016 nan nan
12 2013 自动 2.0
15 2016 nan nan
75 2015 手动 1.5
3 2010 nan 2.5
1 2010 手动 1.4
1 2014 手动 1.6
1 2015 自动 1.4
84 2018 nan 1.6
33 2014 自动 nan
62 2015 nan 1.4
19 2017 自动 1.5
15 2012 nan nan
15 2011 nan nan
95 2010 自动 2.0
36 2017 nan nan
38 2011 nan 1.6
84 2012 手动 1.5
81 2017 自动 1.5
38 2015 nan nan
76 2016 手动 2.0
72 2015 手动 1.6
1 2008 自动 1.6
19 2014 自动 1.8
1 2011 nan 1.8
53 2014 手动 1.6
76 2015 手动 1.6
76 2018 自动 1.6
8 2012 手动 1.6
33 2016 nan nan
3 2018 自动 1.8
45 2014 nan 1.0
27 2012 手动 1.3
76 2015 手动 1.4
67 2017 自动 nan
1 2013 自动 1.6
67 2009 自动 1.8
38 2015 自动 nan
12 2014 自动 1.8
12 2015 自动 2.0
76 2014 nan 1.4
33 2017 nan nan
78 2015 手动 2.0
68 2019 nan 2.4
1 2015 手动 1.6
13 2014 手动 1.6
14 2017 自动 1.5
33 2010 nan 2.0
12 2014 自动 1.8
71 2015 nan 1.5
71 2015 nan 2.4
36 2015 nan 

...

82 2015 自动 nan
8 2013 自动 1.5
1 2015 手动 nan
8 2015 nan 1.5
38 2016 自动 nan
14 2017 nan nan
14 2017 自动 1.5
3 2014 自动 1.6
1 2017 nan nan
1 2013 自动 1.6
3 2014 手动 1.3
181 2017 nan 2.0
63 2016 nan 2.5
38 2012 nan 2.0
33 2012 nan nan
75 2015 nan 1.0
44 2014 nan 2.0
3 2012 nan 2.5
52 2018 nan nan
63 2009 自动 2.0
33 2010 自动 2.0
3 2008 nan 1.6
76 2014 手动 2.0
70 2018 nan nan
8 2016 nan nan
33 2011 nan 2.4
14 2017 自动 1.8
122 2015 自动 1.6
3 2014 自动 1.5
63 2008 自动 1.6
8 2017 自动 1.5
63 2016 自动 1.6
122 2018 自动 2.0
58 2015 自动 2.5
75 2009 nan 1.5
3 2016 自动 1.5
181 2015 手动 1.5
114 2015 nan 1.5
38 2018 自动 nan
8 2013 手动 1.8
114 2015 nan 1.5
20 2018 自动 nan
83 2015 nan 1.5
283 2017 nan 2.0
63 2016 自动 1.6
44 2018 nan 2.0
71 2012 nan 1.6
25 2018 自动 1.8
15 2018 nan nan
3 2011 自动 1.6
120 2013 手动 1.5
8 2011 nan 2.3
84 2011 nan 2.0
40 2016 nan 2.0
8 2017 自动 1.5
165 2017 自动 1.6
181 2016 手动 1.5
26 2016 手动 1.5
13 2014 自动 1.6
14 2017 自动 1.5
58 2008 nan 1.6
71 2018 自动 nan
73 2016 nan 2.0
12 2016 自动 1.6
1 2012 自动 1.4
33 20

36 2015 nan nan
33 2013 nan nan
33 2012 nan nan
76 2012 手动 1.5
3 2017 自动 1.5
76 2014 手动 1.4
57 2009 nan 4.7
33 2014 nan nan
76 2016 自动 1.6
36 2017 nan nan
1 2018 自动 nan
62 2012 nan 1.4
49 2017 nan 2.0
62 2011 nan 1.6
46 2017 nan 2.4
38 2011 nan 1.6
76 2014 手动 1.6
1 2013 自动 2.0
13 2014 手动 1.6
8 2011 nan 2.0
1 2011 手动 1.6
63 2016 自动 1.6
72 2018 自动 1.6
76 2017 nan 1.5
67 2010 自动 1.6
71 2013 手动 1.2
71 2013 手动 1.2
33 2015 nan nan
1 2012 nan 3.0
122 2015 手动 1.6
63 2012 手动 1.6
38 2016 自动 nan
13 2014 手动 1.6
47 2016 nan nan
181 2013 手动 2.0
8 2012 自动 1.6
36 2018 nan nan
75 2016 手动 1.5
70 2014 nan nan
19 2014 手动 1.5
8 2012 手动 1.6
36 2012 nan nan
72 2013 手动 1.6
1 2014 手动 1.6
15 2018 nan nan
38 2017 自动 nan
19 2018 自动 1.5
71 2017 自动 1.5
259 2017 自动 1.5
1 2018 自动 nan
114 2013 nan 1.5
46 2014 nan 2.4
46 2017 自动 nan
15 2013 nan nan
1 2018 自动 nan
3 2013 自动 1.6
71 2015 nan 1.5
12 2014 自动 1.8
38 2017 自动 nan
15 2017 nan nan
70 2016 nan nan
63 2012 nan 1.6
26 2016 手动 1.5
1 2012 nan 1.4
20 2018 自动 1.5
33 201

38 2011 nan 1.6
56 2014 nan 1.2
3 2016 自动 1.6
8 2013 nan 2.0
162 2012 自动 1.8
33 2013 自动 nan
84 2015 nan 1.6
15 2012 nan nan
1 2015 自动 1.6
1 2013 nan 1.8
14 2017 nan nan
1 2011 自动 1.6
8 2018 自动 1.6
58 2014 自动 2.0
1 2013 手动 1.6
58 2010 手动 1.6
14 2015 自动 1.8
67 2011 手动 1.6
1 2016 nan nan
78 2017 自动 2.0
84 2015 自动 1.5
45 2015 nan 1.0
70 2013 nan nan
46 2017 nan 2.0
62 2011 nan 1.6
38 2015 自动 nan
8 2011 自动 1.5
3 2015 nan 2.0
38 2016 自动 nan
8 2013 nan 1.5
67 2016 手动 1.6
72 2013 自动 2.0
1 2015 自动 1.6
76 2015 手动 1.6
12 2017 自动 1.8
1 2013 自动 1.6
13 2014 手动 1.6
165 2014 nan 1.6
62 2015 自动 2.0
14 2015 自动 1.5
72 2015 自动 1.2
1 2011 nan 1.8
67 2018 自动 nan
8 2015 自动 1.6
8 2013 nan 1.6
8 2015 自动 1.6
120 2016 手动 1.5
8 2015 自动 1.5
72 2011 手动 2.0
71 2010 自动 1.4
1 2016 自动 1.4
1 2018 自动 nan
14 2013 nan 2.4
15 2014 nan nan
38 2015 自动 nan
36 2015 nan nan
67 2018 自动 nan
36 2017 nan nan
72 2015 自动 1.6
8 2011 手动 1.8
38 2016 nan nan
114 2016 nan 1.5
58 2014 自动 1.5
1 2015 手动 1.6
1 2016 自动 nan
14 2018 自动 1.5
12 201

26 2014 手动 1.6
1 2014 自动 1.6
8 2013 手动 1.5
3 2011 手动 1.6
1 2016 自动 1.4
1 2014 自动 1.6
1 2012 自动 1.6
81 2017 自动 1.5
181 2013 手动 2.0
8 2018 nan nan
77 2015 自动 1.5
8 2012 自动 1.6
36 2016 nan nan
1 2015 nan 2.0
181 2017 自动 2.0
45 2015 nan 1.0
25 2016 自动 1.8
3 2013 手动 1.6
94 2016 自动 1.5
25 2014 自动 2.0
3 2018 nan 2.0
82 2015 自动 nan
15 2009 nan nan
62 2014 自动 1.8
95 2017 手动 1.6
1 2013 手动 1.6
38 2012 nan 2.4
26 2013 自动 1.6
1 2009 手动 1.6
72 2012 手动 1.6
14 2016 自动 nan
72 2010 自动 2.3
76 2017 手动 1.5
49 2013 nan 2.0
79 2017 自动 1.5
259 2017 手动 1.8
3 2017 自动 1.8
181 2017 自动 1.5
46 2014 nan 2.4
3 2017 自动 1.6
72 2011 自动 2.3
62 2014 自动 1.6
95 2015 自动 2.0
1 2014 自动 1.6
3 2016 自动 2.0
12 2016 自动 1.6
12 2015 自动 1.6
62 2012 自动 2.0
71 2014 自动 2.0
36 2014 nan nan
63 2017 自动 2.0
36 2013 nan nan
58 2014 nan 2.0
14 2013 自动 1.8
36 2015 nan nan
13 2014 自动 1.8
1 2019 自动 1.5
71 2014 自动 2.0
71 2016 自动 1.5
94 2017 自动 2.0
95 2016 自动 2.0
1 2017 nan 2.0
75 2013 自动 1.5
25 2016 自动 1.5
80 2017 手动 1.8
52 2012 nan nan
1 2018 自动 

40 2011 nan 3.0
25 2014 手动 1.5
8 2012 手动 1.6
63 2016 自动 1.6
120 2017 手动 1.5
46 2017 nan 2.0
181 2016 自动 1.5
1 2011 nan 3.0
181 2015 手动 1.5
12 2017 自动 1.6
22 2017 手动 1.5
8 2013 nan 3.5
38 2016 自动 nan
122 2015 自动 2.0
75 2012 手动 1.5
1 2017 nan nan
68 2018 nan 2.0
38 2012 nan 2.4
62 2013 手动 1.6
13 2017 nan nan
14 2012 nan 2.0
120 2016 手动 1.5
75 2018 手动 1.5
76 2017 自动 1.6
67 2013 手动 1.4
114 2013 nan 1.0
8 2013 nan 2.0
1 2018 自动 nan
95 2009 手动 1.6
8 2015 nan 2.0
173 2015 手动 1.5
94 2014 手动 1.5
63 2012 nan 2.0
15 2014 nan nan
181 2009 nan 2.5
80 2016 手动 1.5
49 2017 nan 2.0
53 2009 自动 1.0
26 2017 自动 1.5
73 2015 nan 2.0
52 2016 nan nan
1 2011 nan 2.0
1 2016 手动 1.6
62 2016 自动 2.0
181 2014 手动 1.5
52 2015 nan nan
25 2016 nan 2.4
8 2016 nan nan
1 2015 自动 1.4
86 2014 手动 1.6
8 2013 nan 2.0
181 2017 自动 2.0
1 2013 手动 1.6
14 2014 自动 1.8
181 2011 手动 2.0
1 2015 自动 1.4
114 2015 nan 1.5
13 2011 自动 2.3
38 2014 nan 2.4
38 2014 自动 1.4
71 2015 手动 1.5
53 2011 手动 1.6
1 2018 自动 1.5
1 2017 自动 nan
68 2016 nan 2.4
1 2

In [59]:
# coding=utf-8
import os
import re
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_PINGJIA_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/pingjia?charset=utf8'

engine = create_engine(PRODUCE_PINGJIA_ENGINE, encoding=ENCODING)  

car_source = pd.read_csv('./finish_match.csv')
with engine.begin() as con:
    insert_car_source = car_source.drop(['source_id','online_year','car_image','description','conditions','contact','region','company_name',
                'maintenance','insurance_money','quality_assurance','transfer_owner','annual_insurance',
                'compulsory_insurance','business_insurance','price_bn'],axis=1)
    cur_time = datetime.datetime.now()
    for i in range(0, len(insert_car_source)):
        try:
            url = insert_car_source.loc[i, 'url']
            source = insert_car_source.loc[i:i,:]
            sql = 'delete from car_source where url=\'' +url+ '\' '
            con.execute(sql)
            con.execute('commit')
            source.to_sql(name='car_source', if_exists='append', con=engine, index=False)

            sql = 'select id,thumbnail from car_source where url=\'' +url+ '\' '
            query_car_source = pd.read_sql_query(sql, engine)
            final_car_source = car_source.loc[i:i,:].merge(query_car_source,how='left',on=['thumbnail'])
            car_image = pd.DataFrame()
            car_image['image'] = pd.Series(eval(final_car_source.loc[0,'car_image']))
            car_id = final_car_source.loc[0,'id']
            car_image['car_id'] = car_id
            car_image['create_time'] = cur_time
            sql = 'delete from car_image where car_id = ' + str(car_id)
            con.execute(sql)
            con.execute('commit')
            car_image.to_sql(name='car_image', if_exists='append', con=engine, index=False)

            car_detail_info = final_car_source.copy()
            car_detail_info = car_detail_info.loc[:,['conditions','description','contact','region','company_name','transfer_owner','maintenance','insurance_money','quality_assurance','id','price_bn']]
            car_detail_info['update_time'] = cur_time.strftime('%Y-%m-%d %H:%M:%S')
            car_detail_info['condition_score'] = ''
            car_detail_info = car_detail_info.rename(columns={'id':'car_id','annual_insurance':'examine_insurance','compulsory_insurance':'mandatory_insurance',
                                                             'conditions':'condition'})
            car_detail_info['mdn_status'] = 'P'
            car_detail_info['dealer_status'] = 0

            sql = 'delete from car_detail_info where car_id = ' + str(car_id)
            con.execute(sql)
            con.execute('commit')
            car_detail_info.to_sql(name='car_detail_info', if_exists='append', con=engine, index=False)
            print i
        except Exception as e:
            print e.message
con.close()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42858 entries, 0 to 42857
Data columns (total 50 columns):
source_id               42858 non-null int64
url                     42858 non-null object
title                   42858 non-null object
year                    42858 non-null int64
month                   42855 non-null float64
mile                    42858 non-null float64
city                    42858 non-null object
color                   0 non-null float64
phone                   42712 non-null object
thumbnail               42858 non-null object
price                   42858 non-null float64
domain                  42858 non-null object
transfer_owner          0 non-null float64
annual_insurance        22824 non-null object
compulsory_insurance    0 non-null float64
business_insurance      21880 non-null object
description             41445 non-null object
conditions              0 non-null float64
contact                 42858 non-null object
region                  3638

### 二.估值数据库更新脚本

In [2]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

reload(sys)
sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_USED_CAR_ESTIMATE_ENGINE, encoding=ENCODING)  

global_model_mean = pd.read_csv('./global_model_mean.csv')
global_model_mean = global_model_mean.drop(['listed_year'], axis=1)

with engine.begin() as con:
    sql = 'delete from valuate_global_model_mean where update_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' ' 
    con.execute(sql)
    con.execute('commit')
con.close()

global_model_mean.to_sql(name='valuate_global_model_mean', if_exists='append', con=engine, index=False)

'2019.1'

In [ ]:
# coding=utf-8
import os
import re
import sys
import json
import math
import random
import datetime
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

reload(sys)
sys.setdefaultencoding('utf8')

ENCODING = 'utf-8'
PRODUCE_DB_ADDR_OUTTER = '10.45.138.200'
PRODUCE_DB_USER = 'pingjia'
PRODUCE_DB_PASSWD = 'De32wsxC'
PRODUCE_USED_CAR_ESTIMATE_ENGINE = 'mysql+pymysql://'+PRODUCE_DB_USER+':'+PRODUCE_DB_PASSWD+'@'+PRODUCE_DB_ADDR_OUTTER+'/china_used_car_estimate?charset=utf8'
engine = create_engine(PRODUCE_USED_CAR_ESTIMATE_ENGINE, encoding=ENCODING)

##########################################################
car_autohome_cos_vector = pd.read_csv('./car_autohome_cos_vector.csv')

with engine.begin() as con:
    sql = 'delete from match_cos_vector where create_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' '
    con.execute(sql)
    con.execute('commit')
con.close()
car_autohome_cos_vector.to_sql(name='match_cos_vector', if_exists='append', con=engine, index=False)

##########################################################
brand_name = pd.read_csv('./brand_name.csv')

with engine.begin() as con:
    sql = 'delete from match_brand_name where create_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' '
    con.execute(sql)
    con.execute('commit')
con.close()
brand_name.to_sql(name='match_brand_name', if_exists='append', con=engine, index=False)

##########################################################
word_index = pd.read_csv('./word_index.csv')

with engine.begin() as con:
    sql = 'delete from match_word_index where create_time < \''+  datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") +'\' '
    con.execute(sql)
    con.execute('commit')
con.close()
word_index.to_sql(name='match_word_index', if_exists='append', con=engine, index=False)


In [ ]:
import time
import pandas as pd
import numpy as np

import os
os.environ['VALUATE_RUNTIME_ENVIRONMENT'] = 'INTRANET'

from match.predict.predict_api import Predict

time1 = time.time()
print('start project!')
result = pd.DataFrame()
car_source = pd.read_csv('./wait_match.csv')
detail_name = car_source.loc[:, ['detail_name']].sort_values(by=['detail_name'])
detail_name = detail_name.drop_duplicates(['detail_name']).reset_index(drop=True)

print('等待匹配车源数据量:', len(car_source), '款型数量:', len(detail_name))
match = Predict()
for i, detail in enumerate(list(detail_name.loc[:, 'detail_name'].values)):

    part1 = car_source.loc[(car_source['detail_name'] == detail), :].reset_index(drop=True)
    print(i, detail_name['detail_name'][i], len(part1))
    gpj_detail = match.predict(part1['detail_name'][0], cos_similar=0.81)['data']
    print(gpj_detail)
    if len(gpj_detail) == 0:
        part1['origin_name'] = part1['detail_name']
        part1 = part1.drop(['detail_name'], axis=1)
        result = result.append(part1, sort=False).reset_index(drop=True)
    else:
        part2 = pd.DataFrame(gpj_detail, index=[0])
        part1['origin_name'] = part1['detail_name']
        part1 = part1.drop(['detail_name'], axis=1)
        temp = pd.concat([part1, pd.DataFrame(columns=list(part2.columns))], sort=False)
        temp.loc[:, part2.columns] = part2.loc[0, :].values
        result = result.append(temp, sort=False).reset_index(drop=True)

result = result.loc[:, ['origin_name', 'brand_name', 'model_name', 'detail_name', 'cos_similar', 'brand_slug', 'model_slug', 'detail_slug', 'online_year',
                        'energy', 'body', 'control', 'volume', 'year', 'month', 'mile', 'city', 'price_bn', 'price', 'create_time', 'domain', 'labels', 'url']]
result.to_csv('./train_temp.csv', index=False)
time2 = time.time()
print('cost time', time2-time1)

In [4]:
def process_price_bn(df):
    if df['price_bn'] == '暂无报价':
        return np.NAN
    temp = re.findall(r'万',df['price_bn'])
    if len(temp) > 1:
        text = df['price_bn'].split('~')[1]
    else:
        text = df['price_bn']
    text = text.replace('万','')
    return float(text)

def delete_str_useless(df, column_name):
    """
    删除没用的字符
    """    
    text = df[column_name]
    text = text.replace('·', '')
    text = text.replace('・', '')
    text = text.replace('-', '')
    text = text.replace(' ', '')
    text = text.lower()
    return text

# 判断是否有重复final_text
origin_train = pd.read_csv('../tmp/train/wait_match.csv')

origin_train = origin_train.loc[(origin_train['detail_name'].notnull())&(origin_train['price'] > 0), :].reset_index(drop=True)
origin_train['title'] = origin_train.apply(delete_str_useless, args=('detail_name',), axis=1)
origin_train = origin_train.rename(columns={'detail_name':'origin_name'})
train_final = pd.read_csv('../tmp/train/train_final.csv')
train_final = train_final.rename(columns={'final_text':'title'})

In [5]:
# data = wait_predict.loc[(wait_predict['domain'].isin(['guazi.com'])), :]
data = origin_train.copy()
print('瓜子车源数据量:', len(data))
final = data.merge(train_final, how='left', on=['title'])
final = final.sort_values(by=['title'])
match = final.loc[(final['online_year'].notnull())&(final['city'].notnull()), :].reset_index(drop=True)
print('瓜子车源数据匹配量:',len(match), '匹配率:', len(match)/len(data))
match.info()

瓜子车源数据量: 286288
瓜子车源数据匹配量: 268593 匹配率: 0.9381916112446208
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268593 entries, 0 to 268592
Data columns (total 19 columns):
url            268593 non-null object
origin_name    268593 non-null object
year           268496 non-null float64
month          268496 non-null float64
mile           268593 non-null float64
city           268593 non-null object
price          268593 non-null float64
domain         268593 non-null object
labels         254874 non-null object
create_time    268593 non-null object
title          268593 non-null object
brand_slug     268593 non-null float64
model_slug     268593 non-null float64
detail_slug    268593 non-null float64
online_year    268593 non-null float64
price_bn       268593 non-null float64
brand_name     268593 non-null object
model_name     268593 non-null object
detail_name    268593 non-null object
dtypes: float64(9), object(10)
memory usage: 38.9+ MB


In [7]:
old_version = match.copy()
old_version = old_version.loc[:, ['url','origin_name','detail_slug']].rename(columns={'detail_slug':'old_detail_slug'})
old_version = old_version.drop_duplicates(['url']).reset_index(drop=True)
train_temp = pd.read_csv('../tmp/train/train_temp.csv')
train_temp = train_temp.drop_duplicates(['url']).reset_index(drop=True)
finish = old_version.merge(train_temp,how='left',on=['url'])
test = finish.loc[(finish['old_detail_slug'] != finish['detail_slug']),:]
test = test.drop_duplicates(['origin_name_x']).reset_index(drop=True)
test

,url,origin_name_x,old_detail_slug,origin_name_y,brand_name,model_name,detail_name,cos_similar,brand_slug,model_slug,...,volume,year,month,mile,city,price_bn,price,create_time,domain,labels
0,https://www.guazi.com/sz/3008413906x.htm,DS 4 2012款 1.6T 雅致版(进口),7256.0,DS 4 2012款 1.6T 雅致版(进口),DS,DS 5(进口),2012款 1.6T 雅致版,0.952,169.0,2314.0,...,1.6,2014.0,7.0,2.7,深圳,29.88,12.50,2019-03-19 00:42:19,guazi.com,"急售,质保"
1,https://www.guazi.com/jiamusi/3008290892x.htm,Jeep 自由侠 2016款 1.4T 自动劲能版+,26537.0,Jeep 自由侠 2016款 1.4T 自动劲能版+,Jeep,自由侠,2016款 1.4T 自动劲能版,1.000,46.0,4072.0,...,1.4,2016.0,9.0,4.0,佳木斯,15.88,13.00,2019-03-12 13:35:41,guazi.com,"0过户,质保"
2,https://www.guazi.com/cs/3008516485x.htm,MINI 2014款 1.2T ONE,18991.0,MINI 2014款 1.2T ONE,MINI,MINI,2014款 1.2T ONE+,1.000,56.0,209.0,...,1.2,2015.0,1.0,3.5,长沙,22.50,13.00,2019-03-22 10:54:33,guazi.com,"全国购,4s店保养,质保"
3,https://www.guazi.com/jinzhou/3008319254x.htm,中华H320 2012款 1.5L 手动舒适型,13734.0,中华H320 2012款 1.5L 手动舒适型,中华,中华H230,2012款 1.5L 手动舒适型,1.000,22.0,2770.0,...,1.5,2016.0,2.0,2.3,锦州,5.58,2.80,2019-03-14 23:41:05,guazi.com,质保
4,https://www.guazi.com/hengshui/3005752416x.htm,华泰EV160R 2017款 标准型,31081.0,华泰EV160R 2017款 标准型,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2017.0,12.0,1.9,衡水,NaN,3.41,2019-03-05 08:49:02,guazi.com,急售
5,https://www.guazi.com/bj/3008236007x.htm,启辰 晨风 2014款 领航版6.6KW,19922.0,启辰 晨风 2014款 领航版6.6KW,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2015.0,7.0,2.0,北京,NaN,6.00,2019-03-04 18:24:46,guazi.com,0过户
6,https://www.guazi.com/dl/3007948232x.htm,启辰 晨风 2017款 领航极地版6.6KW,28622.0,启辰 晨风 2017款 领航极地版6.6KW,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2016.0,12.0,3.9,大连,NaN,10.00,2019-03-04 18:24:55,guazi.com,超值
7,https://www.guazi.com/daqing/3006123916x.htm,大众 蔚领 2017款 1.6L 手动舒适型,28272.0,大众 蔚领 2017款 1.6L 手动舒适型,大众,速腾,2017款 1.6L 手动舒适型,0.900,1.0,442.0,...,1.6,2016.0,11.0,3.3,大庆,13.88,9.20,2019-03-05 02:29:54,guazi.com,"4s店保养,0过户"
8,https://www.guazi.com/wx/3008370669x.htm,大众 蔚领 2017款 1.6L 自动舒适型,28273.0,大众 蔚领 2017款 1.6L 自动舒适型,大众,速腾,2017款 1.6L 自动舒适型,0.900,1.0,442.0,...,1.6,2017.0,3.0,2.1,无锡,15.08,11.46,2019-03-15 01:36:26,guazi.com,质保
9,https://www.guazi.com/leshan/3003301372x.htm,大众蔚领 2017款 1.6L 自动舒适型,28273.0,大众蔚领 2017款 1.6L 自动舒适型,大众,速腾,2017款 1.6L 自动舒适型,0.900,1.0,442.0,...,1.6,2017.0,9.0,0.9,乐山,15.08,10.80,2019-03-05 02:30:21,guazi.com,"4s店保养,0过户"


In [ ]:
#----------------
# 1.车系去掉品牌
# 2.1.5L去掉L